#### Importações

In [73]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [74]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [75]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)', 'impacts_of_option-partial']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [76]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [77]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,125.587400,22.307377,22.340637,99.000,99.000,107.500,1.020000e+02,1.160000e+02,1.220000e+02,142.500000,152.000000,1.560000e+02,1.560000e+02
weight (t),18815.714286,18514.786300,5472.264453,5796.655382,10054.000,10054.000,16530.500,1.277275e+04,1.858400e+04,1.871550e+04,21182.000000,24047.000000,2.764700e+04,2.763400e+04
installation_date,38.857143,37.291700,5.639993,5.880516,31.000,31.000,34.500,3.100000e+01,4.100000e+01,3.600000e+01,42.500000,44.000000,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.449600,0.786796,0.739266,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.780800,1.511858,1.841383,4.000,4.000,8.000,4.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00
number_of_piles,17.142857,16.140100,10.106575,10.526802,5.000,5.000,8.500,5.000000e+00,2.000000e+01,1.300000e+01,23.000000,28.000000,3.200000e+01,3.200000e+01
height_of_jacket_or_sub-structure (m),137.500000,142.081508,18.993420,14.429716,114.000,114.211,122.750,1.297880e+02,1.360000e+02,1.410095e+02,150.500000,155.199000,1.660000e+02,1.659960e+02
distance_to_coast (km),218.571429,225.101700,66.271807,59.263091,120.000,120.000,180.000,1.680000e+02,2.400000e+02,2.520000e+02,264.000000,279.000000,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.454000,0.534522,0.497904,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,2.424791e-10,7.800000e-08,8.457021e-08,0.000012,0.000014,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [78]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [79]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [80]:
df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

0


In [81]:
df_gan_limpo.shape

(10000, 25)

#### Removendo outliers com base na amplitude interquartil

In [82]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [83]:
#df_gan_limpo = remover_outliers_iqr(df_gan)

In [84]:
df_gan.shape, df_gan_limpo.shape

((10000, 25), (10000, 25))

In [85]:
#df_gan_limpo.duplicated().sum()

#### Comparando novamente as bases de dados

In [86]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan_limpo.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [87]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,125.587400,22.307377,22.340637,99.000,99.000,107.500,1.020000e+02,1.160000e+02,1.220000e+02,142.500000,152.000000,1.560000e+02,1.560000e+02
weight (t),18815.714286,18514.786300,5472.264453,5796.655382,10054.000,10054.000,16530.500,1.277275e+04,1.858400e+04,1.871550e+04,21182.000000,24047.000000,2.764700e+04,2.763400e+04
installation_date,38.857143,37.291700,5.639993,5.880516,31.000,31.000,34.500,3.100000e+01,4.100000e+01,3.600000e+01,42.500000,44.000000,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.449600,0.786796,0.739266,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.780800,1.511858,1.841383,4.000,4.000,8.000,4.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00
number_of_piles,17.142857,16.140100,10.106575,10.526802,5.000,5.000,8.500,5.000000e+00,2.000000e+01,1.300000e+01,23.000000,28.000000,3.200000e+01,3.200000e+01
height_of_jacket_or_sub-structure (m),137.500000,142.081508,18.993420,14.429716,114.000,114.211,122.750,1.297880e+02,1.360000e+02,1.410095e+02,150.500000,155.199000,1.660000e+02,1.659960e+02
distance_to_coast (km),218.571429,225.101700,66.271807,59.263091,120.000,120.000,180.000,1.680000e+02,2.400000e+02,2.520000e+02,264.000000,279.000000,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.454000,0.534522,0.497904,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,2.424791e-10,7.800000e-08,8.457021e-08,0.000012,0.000014,2.300000e-05,2.300000e-05


In [88]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),-1.158829e+00,-3.326070e-02,0.000,5.500000e+00,-6.000000e+00,-9.500000,0.000
weight (t),3.009280e+02,-3.243909e+02,0.000,3.757750e+03,-1.315000e+02,-2865.000000,13.000
installation_date,1.565443e+00,-2.405228e-01,0.000,3.500000e+00,5.000000e+00,-1.500000,0.000
type_of_production (1 oil and gas; 2 oil; 3 gas),1.218286e-01,4.752961e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
number_of_legs,6.477714e-01,-3.295249e-01,0.000,4.000000e+00,0.000000e+00,0.000000,0.000
number_of_piles,1.002757e+00,-4.202267e-01,0.000,3.500000e+00,7.000000e+00,-5.000000,0.000
height_of_jacket_or_sub-structure (m),-4.581508e+00,4.563704e+00,-0.211,-7.038000e+00,-5.009500e+00,-4.699000,0.004
distance_to_coast (km),-6.530271e+00,7.008716e+00,0.000,1.200000e+01,-1.200000e+01,-15.000000,0.000
risk_to_other_users-complete,-2.542857e-02,3.661808e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
risk_to_other_users-partial,4.638393e-07,4.126151e-08,0.000,-2.424791e-10,-6.570214e-09,-0.000002,0.000


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [89]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [90]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [91]:
df_gan_limpo.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,126,27620,45,1,8,31,145.164,279,0,5.524069e-09,0.109,0.031,1150311,821970,89667,62958,0.904,0.488,0.999,0.998,0.111,0.983,0.992,1.000,0.529
1,100,26520,33,1,8,9,128.737,280,1,6.306171e-11,0.302,0.125,1148740,723580,90046,62867,0.000,0.009,0.886,0.568,0.048,0.421,0.145,1.000,0.444
2,154,26944,45,2,8,31,162.092,190,0,2.115827e-05,0.031,0.014,566052,664704,37969,51622,1.000,0.499,1.000,0.999,0.685,0.996,0.994,0.987,0.666
3,155,23491,45,3,8,31,164.307,139,0,2.294078e-05,0.026,0.014,313521,478265,24953,36572,0.998,0.498,1.000,0.999,0.887,0.995,0.992,0.621,0.827
4,142,12168,31,1,4,5,145.769,279,1,2.025309e-08,0.158,0.052,392536,436181,26851,33375,0.001,0.000,1.000,1.000,0.925,0.833,0.510,1.000,0.422


In [92]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# # Criar o metadata
# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=A)
# metadata_dict = metadata.to_dict()
metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "risk_to_other_users-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_other_users-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "impacts_of_option-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 25/25 [00:00<00:00, 650.87it/s]|
Column Shapes Score: 74.11%

(2/2) Evaluating Column Pair Trends: |██████████| 300/300 [00:01<00:00, 242.43it/s]|
Column Pair Trends Score: 93.41%

Overall Score (Average): 83.76%



In [93]:
report.get_score()

0.837567492639895

In [94]:
report.get_properties()

,Property,Score
0,Column Shapes,0.741067
1,Column Pair Trends,0.934068


In [95]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,water_depth (m),KSComplement,0.813657
1,weight (t),KSComplement,0.770757
2,installation_date,KSComplement,0.765157
3,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.871529
4,number_of_legs,KSComplement,0.838057
5,number_of_piles,KSComplement,0.767957
6,height_of_jacket_or_sub-structure (m),KSComplement,0.671129
7,distance_to_coast (km),KSComplement,0.745757
8,risk_to_other_users-complete,KSComplement,0.974571
9,risk_to_other_users-partial,KSComplement,0.647629


In [96]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,water_depth (m),weight (t),CorrelationSimilarity,0.931494,0.242657,0.105646
1,water_depth (m),installation_date,CorrelationSimilarity,0.974530,0.742407,0.691467
2,water_depth (m),type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.883813,0.828860,0.596487
3,water_depth (m),number_of_legs,CorrelationSimilarity,0.996163,0.166611,0.174285
4,water_depth (m),number_of_piles,CorrelationSimilarity,0.947475,0.773687,0.668636
5,water_depth (m),height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.970335,0.980269,0.920939
6,water_depth (m),distance_to_coast (km),CorrelationSimilarity,0.980947,-0.592407,-0.554302
7,water_depth (m),risk_to_other_users-complete,CorrelationSimilarity,0.957504,-0.646966,-0.561973
8,water_depth (m),risk_to_other_users-partial,CorrelationSimilarity,0.989129,0.824868,0.803125
9,water_depth (m),risk_to_personnel-complete,CorrelationSimilarity,0.992811,-0.649726,-0.664104
